In [2]:
!pip install datasets
!pip install pydub

In [44]:
from transformers import AutoProcessor, UniSpeechModel, UniSpeechConfig
import torch
from datasets import load_dataset
import numpy as np
import pandas as pd
import requests
from io import BytesIO
from moviepy.editor import VideoFileClip
import librosa
import os
import moviepy.editor
from pydub import AudioSegment
import matplotlib.pyplot as plt
from scipy.signal import stft
from scipy.io import wavfile
import scipy.signal as sps
import torch.nn as nn
import gc
import re

In [4]:
processor = AutoProcessor.from_pretrained("patrickvonplaten/unispeech-large-1500h-cv-timit")
model_pret = UniSpeechModel.from_pretrained("patrickvonplaten/unispeech-large-1500h-cv-timit")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of UniSpeechModel were not initialized from the model checkpoint at patrickvonplaten/unispeech-large-1500h-cv-timit and are newly initialized: ['unispeech.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'unispeech.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
print(model_pret)

UniSpeechModel(
  (feature_extractor): UniSpeechFeatureEncoder(
    (conv_layers): ModuleList(
      (0): UniSpeechLayerNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (1-4): 4 x UniSpeechLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (5-6): 2 x UniSpeechLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): UniSpeechFeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=1024, bias=True)
    (dr

In [121]:
configuration = UniSpeechConfig(vocab_size=31,num_attention_heads=16,hidden_size=1024,conv_bias=True,feat_extract_norm = "layer",num_hidden_layers=12,conv_dim=(512, 512, 512, 512),conv_stride=(5, 2, 2, 2), conv_kernel=(10, 3, 3, 3))

In [122]:
model = UniSpeechModel(configuration)

In [123]:
model.config

UniSpeechConfig {
  "activation_dropout": 0.1,
  "apply_spec_augment": true,
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3
  ],
  "conv_stride": [
    5,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.0,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "layerdrop": 0.1,
  "mask_feature_length": 10,
  "mask_feature_min_masks": 0,
  "mask_feature_prob": 0.0,
  "mask_time_length": 10,
  "m

In [124]:
model_pret.config

UniSpeechConfig {
  "_name_or_path": "patrickvonplaten/unispeech-large-1500h-cv-timit",
  "activation_dropout": 0.0,
  "apply_spec_augment": true,
  "architectures": [
    "UniSpeechForCTC"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.0,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_size": 1024,
  "ini

In [125]:
print(model)

UniSpeechModel(
  (feature_extractor): UniSpeechFeatureEncoder(
    (conv_layers): ModuleList(
      (0): UniSpeechLayerNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (1-3): 3 x UniSpeechLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): UniSpeechFeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=1024, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): UniSpeechEncoder(
    (pos_conv_embed): UniSpeechPositionalConvEmbedding(
      (conv): ParametrizedConv1d(
        1024, 1024, kernel_size=(128,), stride=(1,), padding=(64,), gr

In [126]:
model_params=[]
for name,param in model.named_parameters():
  model_params.append(name)
  print(name)
# print(np.array(model_params))

masked_spec_embed
feature_extractor.conv_layers.0.conv.weight
feature_extractor.conv_layers.0.conv.bias
feature_extractor.conv_layers.0.layer_norm.weight
feature_extractor.conv_layers.0.layer_norm.bias
feature_extractor.conv_layers.1.conv.weight
feature_extractor.conv_layers.1.conv.bias
feature_extractor.conv_layers.1.layer_norm.weight
feature_extractor.conv_layers.1.layer_norm.bias
feature_extractor.conv_layers.2.conv.weight
feature_extractor.conv_layers.2.conv.bias
feature_extractor.conv_layers.2.layer_norm.weight
feature_extractor.conv_layers.2.layer_norm.bias
feature_extractor.conv_layers.3.conv.weight
feature_extractor.conv_layers.3.conv.bias
feature_extractor.conv_layers.3.layer_norm.weight
feature_extractor.conv_layers.3.layer_norm.bias
feature_projection.layer_norm.weight
feature_projection.layer_norm.bias
feature_projection.projection.weight
feature_projection.projection.bias
encoder.pos_conv_embed.conv.bias
encoder.pos_conv_embed.conv.parametrizations.weight.original0
encoder

In [127]:
model.feature_projection.layer_norm.weight.shape

torch.Size([512])

In [128]:
dic = {}
with torch.no_grad():
  for name,param in model_pret.named_parameters():
    dic[name] = param

In [129]:
model.feature_extractor.conv_layers[0].conv.weight = dic['feature_extractor.conv_layers.0.conv.weight']
model.feature_extractor.conv_layers[0].conv.bias = dic['feature_extractor.conv_layers.0.conv.bias']
model.feature_extractor.conv_layers[0].layer_norm.weight = dic['feature_extractor.conv_layers.0.layer_norm.weight']
model.feature_extractor.conv_layers[0].layer_norm.bias = dic['feature_extractor.conv_layers.0.layer_norm.bias']

model.feature_extractor.conv_layers[1].conv.weight = dic['feature_extractor.conv_layers.1.conv.weight']
model.feature_extractor.conv_layers[1].conv.bias = dic['feature_extractor.conv_layers.1.conv.bias']
model.feature_extractor.conv_layers[1].layer_norm.weight = dic['feature_extractor.conv_layers.1.layer_norm.weight']
model.feature_extractor.conv_layers[1].layer_norm.bias = dic['feature_extractor.conv_layers.1.layer_norm.bias']

model.feature_extractor.conv_layers[2].conv.weight = dic['feature_extractor.conv_layers.2.conv.weight']
model.feature_extractor.conv_layers[2].conv.bias = dic['feature_extractor.conv_layers.2.conv.bias']
model.feature_extractor.conv_layers[2].layer_norm.weight = dic['feature_extractor.conv_layers.2.layer_norm.weight']
model.feature_extractor.conv_layers[2].layer_norm.bias = dic['feature_extractor.conv_layers.2.layer_norm.bias']

model.feature_extractor.conv_layers[3].conv.weight = dic['feature_extractor.conv_layers.3.conv.weight']
model.feature_extractor.conv_layers[3].conv.bias = dic['feature_extractor.conv_layers.3.conv.bias']
model.feature_extractor.conv_layers[3].layer_norm.weight = dic['feature_extractor.conv_layers.3.layer_norm.weight']
model.feature_extractor.conv_layers[3].layer_norm.bias = dic['feature_extractor.conv_layers.3.layer_norm.bias']

In [130]:
model.feature_projection.layer_norm.weight = dic['feature_projection.layer_norm.weight']
model.feature_projection.layer_norm.bias = dic['feature_projection.layer_norm.bias']
model.feature_projection.projection.weight = dic['feature_projection.projection.weight']
model.feature_projection.projection.bias = dic['feature_projection.projection.bias']

In [131]:
dic['encoder.pos_conv_embed.conv.parametrizations.weight.original0'].shape

torch.Size([1, 1, 128])

In [132]:
model.encoder.pos_conv_embed.conv.bias = dic['encoder.pos_conv_embed.conv.bias']
model.encoder.pos_conv_embed.conv.parametrizations.weight.original0 = dic['encoder.pos_conv_embed.conv.parametrizations.weight.original0']
model.encoder.pos_conv_embed.conv.parametrizations.weight.original1 = dic['encoder.pos_conv_embed.conv.parametrizations.weight.original1']
model.encoder.layer_norm.weight = dic['encoder.layer_norm.weight']
model.encoder.layer_norm.bias = dic['encoder.layer_norm.bias']

for i in range(12):
  model.encoder.layers[i].attention.k_proj.weight = dic['encoder.layers.%d.attention.k_proj.weight'%i]
  model.encoder.layers[i].attention.k_proj.bias = dic['encoder.layers.%d.attention.k_proj.bias'%i]
  model.encoder.layers[i].attention.v_proj.weight = dic['encoder.layers.%d.attention.v_proj.weight'%i]
  model.encoder.layers[i].attention.v_proj.bias = dic['encoder.layers.%d.attention.v_proj.bias'%i]
  model.encoder.layers[i].attention.q_proj.weight = dic['encoder.layers.%d.attention.q_proj.weight'%i]
  model.encoder.layers[i].attention.q_proj.bias = dic['encoder.layers.%d.attention.q_proj.bias'%i]
  model.encoder.layers[i].attention.out_proj.weight = dic['encoder.layers.%d.attention.out_proj.weight'%i]
  model.encoder.layers[i].attention.out_proj.bias = dic['encoder.layers.%d.attention.out_proj.bias'%i]
  model.encoder.layers[i].layer_norm.weight = dic['encoder.layers.%d.layer_norm.weight'%i]
  model.encoder.layers[i].layer_norm.bias = dic['encoder.layers.%d.layer_norm.bias'%i]
  model.encoder.layers[i].feed_forward.intermediate_dense.weight = dic['encoder.layers.%d.feed_forward.intermediate_dense.weight'%i]
  model.encoder.layers[i].feed_forward.intermediate_dense.bias = dic['encoder.layers.%d.feed_forward.intermediate_dense.bias'%i]
  model.encoder.layers[i].feed_forward.output_dense.weight = dic['encoder.layers.%d.feed_forward.output_dense.weight'%i]
  model.encoder.layers[i].feed_forward.output_dense.bias = dic['encoder.layers.%d.feed_forward.output_dense.bias'%i]
  model.encoder.layers[i].final_layer_norm.weight = dic['encoder.layers.%d.final_layer_norm.weight'%i]
  model.encoder.layers[i].final_layer_norm.bias = dic['encoder.layers.%d.final_layer_norm.bias'%i]

In [133]:
for i in range(12):
  print(torch.mean(model.encoder.layers[i].final_layer_norm.bias - model_pret.encoder.layers[i].final_layer_norm.bias))

tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)


In [134]:
model

UniSpeechModel(
  (feature_extractor): UniSpeechFeatureEncoder(
    (conv_layers): ModuleList(
      (0): UniSpeechLayerNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (1-3): 3 x UniSpeechLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): UniSpeechFeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=1024, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): UniSpeechEncoder(
    (pos_conv_embed): UniSpeechPositionalConvEmbedding(
      (conv): ParametrizedConv1d(
        1024, 1024, kernel_size=(128,), stride=(1,), padding=(64,), gr

Inputting the audio now

In [78]:
def download_video(url, save_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"Video downloaded successfully to {save_path}")
    else:
        print(f"Failed to download video. Status code: {response.status_code}")

In [79]:
def delete_file(file_path):
    try:
        os.remove(file_path)
        print(f"File '{file_path}' successfully deleted.")
    except OSError as e:
        print(f"Error deleting file '{file_path}': {e}")

In [80]:
def mp3_to_waveform(mp3_path):
    audio = AudioSegment.from_file(mp3_path, format="mp3").set_frame_rate(16000)
    if audio.channels > 1:
        audio = audio.set_channels(1)
    waveform = np.array(audio.get_array_of_samples())
    return waveform, audio.frame_rate

In [81]:
def getWaveform(vid_url):
    download_video(vid_url, "downloaded_video.mp4")
    video = moviepy.editor.VideoFileClip("downloaded_video.mp4")
    audio = video.audio
    audio.write_audiofile("downloaded_audio.mp3")
    delete_file("downloaded_video.mp4")
    waveform,sample_rate = mp3_to_waveform("downloaded_audio.mp3")
    print("Shape of waveform array:", waveform.shape)
    delete_file("downloaded_audio.mp3")
    return waveform,sample_rate

In [82]:
wave,sample_rate = getWaveform("https://video.twimg.com/amplify_video/1019341984543268866/vid/640x360/ewrbY6gdakX6772y.mp4?tag=3")

Video downloaded successfully to downloaded_video.mp4
MoviePy - Writing audio in downloaded_audio.mp3


MoviePy - Done.
File 'downloaded_video.mp4' successfully deleted.
Shape of waveform array: (924320,)
File 'downloaded_audio.mp3' successfully deleted.


In [83]:
wave.shape,sample_rate

((924320,), 16000)

In [135]:
inputs = processor(wave, sampling_rate=sample_rate, return_tensors="pt")

In [136]:
inputs['input_values']=inputs['input_values'].float()
inputs['input_values']

tensor([[0.0006, 0.0006, 0.0006,  ..., 0.4543, 0.3841, 0.2471]])

In [137]:
gc.collect()

1035

In [138]:
inputs

{'input_values': tensor([[0.0006, 0.0006, 0.0006,  ..., 0.4543, 0.3841, 0.2471]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)}

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
last_hidden_states.shape

In [1]:
with torch.no_grad():
    outputs = model_pret(**inputs)

last_hidden_states = outputs.last_hidden_state
last_hidden_states.shape

NameError: ignored